# PTF_MVT 04: Consolidation Testing

**Purpose**: Test consolidation (AZ + AZEC → Gold)

**Tests**: Union, IRD enrichment, ISIC codification, client data

---

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")

In [ ]:
from pyspark.sql import SparkSession
from azfr_fsspec_utils import fspath
import azfr_fsspec_abfs

azfr_fsspec_abfs.use()

spark = SparkSession.builder \
    .appName("PTF_MVT_Consolidation") \
    .getOrCreate()

print(f"✓ Spark {spark.version}")

## 1. Initialize

In [ ]:
from utils.loaders.config_loader import ConfigLoader
from utils.logger import PipelineLogger

config = ConfigLoader(str(project_root / "config" / "config.yml"))
logger = PipelineLogger("consolidation_test")

VISION = "202509"
print(f"Vision: {VISION}")

## 2. Run Consolidation Processor

In [ ]:
from src.processors.ptf_mvt_processors import ConsolidationProcessor

try:
    consolidation = ConsolidationProcessor(spark, config, logger)
    df_gold = consolidation.run(VISION)
    
    if df_gold is not None:
        print(f"✓ Consolidation: {df_gold.count():,} rows")
        print(f"  Columns: {len(df_gold.columns)}")
    else:
        print("⚠ Consolidation returned None")
except Exception as e:
    print(f"✗ Error: {e}")
    df_gold = None

## 3. Verify Gold Output

In [ ]:
if df_gold is not None:
    # DIRCOM distribution (AZ vs AZEC)
    print("DIRCOM distribution:")
    df_gold.groupBy('dircom').count().show()
    
    # Sample output
    df_gold.select('nopol', 'dircom', 'cdprod', 'primeto').show(5)

## Summary

In [ ]:
print("="*50)
print("CONSOLIDATION TESTING COMPLETE")
print("="*50)
print(f"Result: {'✓ Success' if df_gold is not None else '✗ Failed'}")
print("\n→ Run: python main.py --vision 202509 --component ptf_mvt")